# Scraping User Reviews from PakWheels

In this notebook we scrape user reviews from the PakWheels website using Python libraries such as `requests` and `BeautifulSoup`.

**Imports**

In [4]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import re
import os
from json import loads, dumps
from tqdm import tqdm
from time import sleep

## Scraping User Reviews

A function to scrape user reviews for a car filtered by make and model.

In [34]:
def get_car_reviews(make, model, start=1):
    all_reviews = []
    while True:
        url = f'https://www.pakwheels.com/new-cars/{make}/{model}/reviews/?page={start}'
        response = requests.get(url)
        if response.url != url:
            break
        soup = BeautifulSoup(response.content, 'lxml')
        for div in soup.find('div', class_='well mb20').find_all('div', class_='mb40'):
            review_tagline = div.find('div', class_='col-md-9').find('h3').text.strip()
            car_name = div.find('div', class_='col-md-9').find('a', class_='generic-gray').text.strip()
            meta_info = div.find('div', class_='col-md-9').find('p', class_='date').text.strip()
            try:
                familiarity = div.find('div', class_='col-md-9').find('div', class_='familiarity').text.strip()
            except:
                familiarity = ''
            try:
                review_body = loads(div.find('script').text)['reviewBody'].strip()
            except:
                review_body = ''
            try:
                overall_rating = loads(div.find('script').text)['reviewRating']['ratingValue']
            except:
                overall_rating = float('nan')
            all_ratings = div.find('ul', class_=re.compile(r'review-rating list-unstyled clearfix')).find_all('li')
            style = all_ratings[0].find('img').get('alt').strip()
            comfort = all_ratings[1].find('img').get('alt').strip()
            fuel_economy = all_ratings[2].find('img').get('alt').strip()
            perfomance = all_ratings[3].find('img').get('alt').strip()
            value_for_money = all_ratings[4].find('img').get('alt').strip()
            helpfulness = div.find('div', class_='col-md-6').find_all('div')[-1].text.strip()
            review_obj = {
                'car_model': car_name,
                'review_title': review_tagline,
                'reviewer_info': meta_info,
                'familiarity': familiarity,
                'review_text' : review_body,
                'style': style,
                'comfort_rating': comfort,
                'fuel_economy': fuel_economy,
                'perfomance': perfomance,
                'value_for_money': value_for_money,
                'overall_rating': overall_rating,
                'helpful_votes': helpfulness
            }
            all_reviews.append(review_obj)
        start += 1

    return all_reviews

Utility Functions

In [7]:
def create_directories(make, models):
    for model in models:
        dir = os.path.join('reviews', make, model)
        if not os.path.exists(dir):
            os.makedirs(dir)

In [ ]:
def format_string(input):
    return input.replace(' ', '-').lower()

## Loading Make-Model List and Creating Directories

We will load a list of car makes and models from a JSON file and create directories for each make and its corresponding models.

In [8]:
with open('json/make_model_list.json', 'r') as f:
    make_models_list = loads(f.read())

for idx in range(len(make_models_list)):
    create_directories(make_models_list[idx]['make'], make_models_list[idx]['models'])

In [12]:
for make in os.listdir('reviews'):
    for model in tqdm(os.listdir(os.path.join('reviews', make)), desc=f'Processing {make}'):
        try:
            reviews = get_car_reviews(format_string(make), format_string(model))
            if len(reviews):
                with open(os.path.join('reviews', make, model, 'reviews_list.json'), 'w') as f:
                    f.write(dumps(reviews))
            sleep(5)
        except:
            print(make, model)
            continue

Processing Mitsubishi:  50%|█████     | 12/24 [01:19<00:59,  4.94s/it]

Mitsubishi Minicab


Processing Mitsubishi:  88%|████████▊ | 21/24 [02:21<00:16,  5.36s/it]

Mitsubishi Lancer Evolution X


Processing Ford:  83%|████████▎ | 10/12 [01:03<00:10,  5.12s/it]

...

Processing Mercedes Benz: 100%|██████████| 29/29 [03:07<00:00,  6.46s/it]


Mercedes Benz Brabus


Processing Changan:  81%|████████▏ | 13/16 [01:38<00:17,  5.80s/it]

Changan Kaghan XL


Processing Changan: 100%|██████████| 16/16 [02:01<00:00,  7.60s/it]


**A sample from user reviews data**

In [7]:
df = pd.read_csv('datasets/dataset_2.csv')
df.sample(5)

,make,model,car_model,review_title,reviewer_info,familiarity,review_text,style,comfort_rating,fuel_economy,perfomance,value_for_money,overall_rating,helpful_votes
2544,Hyundai,Elantra,2022 Hyundai Elantra GLS,best available sedan against civic n grande,"Posted by PWUser164991172740 on Mar 26, 2023",Familiarity: I owned this car.,beautiful stylish\ngave me average as high as ...,5 rating,5 rating,5 rating,5 rating,5 rating,5.0,(0 out of 0 people found this review helpful)
569,Prince,Pearl,2022 Prince Pearl MT,Affordable Car of Current Time,"Posted by BILLS on Jul 29, 2023",Familiarity: I owned this car.,I am using this car for more than a year. This...,5 rating,5 rating,3 rating,4 rating,5 rating,4.0,(1 out of 1 person found this review helpful)
1666,Suzuki,Cultus,Suzuki Cultus VXRi (CNG),Moderate,"Posted by Asfand Sadiq on Jan 30, 2012",NaN,Well cultus is an extremly reliable car. \r\nA...,4 rating,3 rating,2 rating,1 rating,5 rating,3.0,(1 out of 3 people found this review helpful)
4595,Toyota,Corolla,Toyota Corolla GLi 1.3 VVTi,Corolla v.s City,"Posted by qwerty2 on May 27, 2011",NaN,I had a city and a corolla i just sold of coro...,3 rating,1 rating,3 rating,4 rating,3 rating,3.0,(6 out of 14 people found this review helpful)
2232,Honda,City,2022 Honda City 1.2L CVT,Great Value,"Posted by Shan on Jun 03, 2022",Familiarity: I owned this car.,"Stylish Exterior, Sporty Look\nDRLs look great...",5 rating,4 rating,4 rating,4 rating,5 rating,4.0,(0 out of 1 person found this review helpful)
